In [26]:
import pickle
from pydantic import BaseModel
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')

class Output(BaseModel):
    content: str
    relevance: float

class Retriever:
    def __init__(self, 
                 path_csv, 
                 path_model):
        
        self.language = None
        if "sat" in path_csv:
            self.language = 'english'
        elif "icfes" in path_csv:
            self.language = 'spanish'
        elif "exani" in path_csv:
            self.language = 'spanish'
        elif "exames_nacionais" in path_csv:
            self.language = 'portuguese'
        elif "cuet" in path_csv:
            self.language = 'english'
        elif "enem" in path_csv:
            self.language = 'portuguese'
        else:
            self.language = 'portuguese'
        
        self.df = pd.read_csv(path_csv)
        self.df['content'] = self.df['content'].apply(self.preprocess_text)
        
        try:
            with open(path_model, 'rb') as f_in:
                self.vectorizer = pickle.load(f_in)
            self.X = self.vectorizer.transform(self.df['content'])    
            
        except:
            print('Model not found. Creating a new one...')
            self.vectorizer = TfidfVectorizer()
            self.X = self.vectorizer.fit_transform(self.df['content'])
            print('Model created.')
            print('Saving model...')
            self.save_model(filename=path_model)
            
    def invoke(self, query, k=3):
        Q = self.vectorizer.transform([query])
        R = self.X.dot(Q.T)
        
        # Get score with np.argsort
        scores = np.array(R.toarray()).flatten()
        idxs = np.argsort(scores)[::-1]
        idxs_and_scores = np.array([[idx, scores[idx]] for idx in idxs])
        
        retrieved = np.array(idxs_and_scores[:k])
        idxs = np.array(retrieved)[:, 0].astype(int)
        output_content = self.df.iloc[idxs][['content']].values
        output_relevance = retrieved[:, 1].reshape(-1, 1)
        output = np.concatenate([output_content, output_relevance], axis=1)
        return output
    
    def query(self, query, k=3):
        output = self.invoke(query, k)
        output = [Output(content=x[0], relevance=x[1]) for x in output]
        return output
    
    def preprocess_text(self, 
                        text):
        
        if self.language == 'portuguese':
            stop_words = set(stopwords.words('portuguese'))
        elif self.language == 'english':
            stop_words = set(stopwords.words('english'))
        elif self.language == 'spanish':
            stop_words = set(stopwords.words('spanish'))
        else:
            stop_words = set()
        
        text = text.lower()
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'\b\w{1,2}\b', '', text)  
        text = re.sub(r'[^\w\s]', '', text) 
        text = ' '.join(word for word in text.split() if word not in stop_words)
        return text
    
    def save_model(self, filename='../vectorstore/tfidf_model.pkl'):
        # Salvar o modelo TF-IDF em um arquivo .pkl
        with open(filename, 'wb') as f_out:
            pickle.dump(self.vectorizer, f_out)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
retrieve_enem = Retriever(path_csv="../vectorstore/data_playlists_enem.csv", 
                          path_model='../vectorstore/tfidf_model_enem.pkl')

retrieve_cuet = Retriever(path_csv="../vectorstore/cuet_edital.csv", 
                          path_model='../vectorstore/tfidf_model_cuet_edital.pkl')

retrieve_exames = Retriever(path_csv="../vectorstore/exames_nacionais_edital.csv", 
                            path_model='../vectorstore/tfidf_model_exames_nacionais_edital.pkl')

retrieve_exani = Retriever(path_csv="../vectorstore/exani_edital.csv", 
                           path_model='../vectorstore/tfidf_model_exani_edital.pkl')

retrieve_icfes = Retriever(path_csv="../vectorstore/icfes_edital.csv", 
                           path_model='../vectorstore/tfidf_model_icfes_edital.pkl')

retrieve_sat = Retriever(path_csv="../vectorstore/sat_edital.csv", 
                         path_model='../vectorstore/tfidf_model_sat_edital.pkl')

In [28]:
query = "Dom João VI Ele veio para cá negociando inclusive com a Inglaterra né para vocês saberem a Marinha britânica que fez a escolta da família real até o Brasil chegaram ao Brasil você vai encontrar em vários livros falando que que você teve problema ali de de desinteria problema de piolho né o pessoal teve de arreia mesmo m né piolho mas na verdade isso tudo aconteceu porque a viagem não era tão rápida e é a primeira vez que uma nobreza portuguesa viaja a sua colônia tá a colônia e a sua colônia mais Próspera pelo menos então eles reclamaram de tudo reclamaram do desconforto da viagem porque a nobreza"

retrieve_enem.query(query, k=5)

[Output(content='title período joanino enem views count publish date author parabólica description unknown link video httpswwwyoutubecomwatchfcr salve salve gente tudo bem bemvindos vídeo parabólica hoje dia videoaula história brasil momento decisivo vamos falar hoje período joanino finalzinho processo colonial brasileiro quer dizer próximas aulas gente vai entrando período império brasil independente vou enrolar gente começa falando sobre período chamado período joanino período aqui brasil vai período joanino simples família real portuguesa veio residir brasil durante período vieram quê vieram porque fugindo napoleão bonaparte imperador francês bom gente fez aula aqui sobre napoleão guerras napoleônicas então assistiu sabe bem acontecendo napoleão decretou bloqueio continental inglat terra nenhuma nação poderia continuar comercializando inglaterra desobedecesse invadido tropas francesas portugal condições condições romper inglaterra porque portugal dependia economicamente inglaterra i

In [29]:
query = """Eligibility 
For appearing in the CUET (UG) - 2025, there is no age limit for the candidates. The candidates 
who have passed the class 12 /equivalent examination or are appearing in 2025 can appear in 
the CUET (UG) - 2025 examination. However, the candidates will be required to fulfill the age 
criteria  (if  any)  of  the  University  /  Institution  /  Organization  in  which  they  are  desirous  of 
taking admission. """

retrieve_cuet.query(query, k=5)

[Output(content='chapter eligibility eligibility appearing cuet age limit candidates candidates passed class equivalent examination appearing appear cuet examination however candidates required fulfill age criteria university institution organization desirous taking admission list qualifying examinations final examination system conducted recognized central state board central board secondary education new delhi council indian school certificate examinations new delhi', relevance=0.6935353949634812),
 Output(content='tests subject covered cuet list languages domain specific subjects general aptitude test covered cuet given appendix duration test duration test paper would minutes xamination conducted multiple shifts depending number candidates combinations compensatory time pwbd candidates minutes hour chapter eligibility eligibility appearing cuet age limit candidates candidates passed class equivalent examination appearing appear', relevance=0.4441758941050143),
 Output(content='marks

In [30]:
query = """Eligibility 
For appearing in the CUET (UG) - 2025, there is no age limit for the candidates. The candidates 
who have passed the class 12 /equivalent examination or are appearing in 2025 can appear in 
the CUET (UG) - 2025 examination. However, the candidates will be required to fulfill the age 
criteria  (if  any)  of  the  University  /  Institution  /  Organization  in  which  they  are  desirous  of 
taking admission. """

retrieve_exames.query(query, k=5)

[Output(content='norma jne instruções realização classificação reapreciação reclamação norma jne instruções realização classificação reapreciação reclamação ficha técnica título norma jne instruções realização classificação reapreciação reclamação provas exames ensino básico ensino secundário autores ana cláudia soeiro clara romano isabel rebelo maria elvira monteiro raquel dionísio ricardo patrão rui ferreira coordenação luís duque almeida capa isabel espinheira composição direçãogeral educação júri nacional exames edição maio', relevance=0.0),
 Output(content='alunos ensino secundário acesso ensino superior reclamação apenas pode incidir sobre questões objeto reapreciação quer alegadas aluno quer tendo sido alegadas mereceram alteração classificação parte professor relator norma jne instruções realização classificação reapreciação reclamação modelo alegação justificativa reclamação continuação número suposto prova processo reclamação preencher jne fundamentação pedido reclamação cont

In [31]:
query = """Eligibility 
For appearing in the CUET (UG) - 2025, there is no age limit for the candidates. The candidates 
who have passed the class 12 /equivalent examination or are appearing in 2025 can appear in 
the CUET (UG) - 2025 examination. However, the candidates will be required to fulfill the age 
criteria  (if  any)  of  the  University  /  Institution  /  Organization  in  which  they  are  desirous  of 
taking admission. """

retrieve_exani.query(query, k=5)

[Output(content='pounds located the final clause the preceding ragraph there the author questions the validity the millerurey experiment the grounds that their theory the origin life would have required methane and monia have been present essential elements for the production amino acids opción argumentación amino acids incorrect according the text amino acids would the result the presence methane and ammonia early earth atmosphere respuesta correcta guía sustentante examen nacional ingreso educación superior exani subárea comprensión lectora tema leer orientarse', relevance=0.6246928892810713),
 Output(content='opción argumentación arrive since incorrect the preposition used this particular context indicate where something someone particularly the case cities countries continents the other hand preposition used indicate position point place which not the case this segment the use the simple present form the verb arrive not appropriate given the time sequence the segment totally the pa

In [32]:
query = """Eligibility 
For appearing in the CUET (UG) - 2025, there is no age limit for the candidates. The candidates 
who have passed the class 12 /equivalent examination or are appearing in 2025 can appear in 
the CUET (UG) - 2025 examination. However, the candidates will be required to fulfill the age 
criteria  (if  any)  of  the  University  /  Institution  /  Organization  in  which  they  are  desirous  of 
taking admission. """

retrieve_icfes.query(query, k=5)

[Output(content='marcarla hoja respuestas presentación especificacionescontenido características desea familiarizarse cinco pruebas examen saber tipos preguntas consulte caja herramientas contiene marcos referencia infografías cuadernillos preguntas ejemplos preguntas explicadas subdirección diseño instrumentos dirección evaluación calle torre piso edificio elemento bogotá colombia wwwicfesgov líneas atención ciudadano bogotá tel pbx', relevance=0.0),
 Output(content='texto evaluado debe seleccionar respuesta correcta cada pregunta cuatro opciones marcarla hoja respuestas base texto evalúa conocimiento gramatical lexical ejercicio consiste elegir palabras adecuadas completar texto ello debe seleccionar palabra correcta decir aquella completa cada espacios texto presentado cuatro opciones dadas marcarla hoja respuestas presentación especificacionescontenido características desea familiarizarse cinco pruebas examen saber tipos preguntas consulte caja', relevance=0.0),
 Output(content='le

In [33]:
query = """Eligibility 
For appearing in the CUET (UG) - 2025, there is no age limit for the candidates. The candidates 
who have passed the class 12 /equivalent examination or are appearing in 2025 can appear in 
the CUET (UG) - 2025 examination. However, the candidates will be required to fulfill the age 
criteria  (if  any)  of  the  University  /  Institution  /  Organization  in  which  they  are  desirous  of 
taking admission. """

retrieve_sat.query(query, k=5)

[Output(content='application considered admission college university complete steps required scholarship programs considered opportunities certain college university systems submit score one school schools within system also access score please note however applying one school within college university system still important send sat scores individual school sure whether specific school applying part system contact school admissions office score reporting', relevance=0.15633010815883164),
 Output(content='entirely whether opt service free education organizations may pay licensing fee use service use fees support nonprofit missiondriven work including providing fee waivers students lower income families take sat free college board nonprofit organization part student search service voluntary opt time contacted education organization mean admitted received scholarship must submit required information complete steps requested education organization potential admission enrollment scholarshi